In [ ]:
## Global Requirements :

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from osgeo import gdal
import sys
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data collection :

## 1- Field Measurements :

In [ ]:
!pip install cartopy

In [ ]:
pip install ismn

In [ ]:
from ismn.interface import ISMN_Interface

In [ ]:
data_path = "/content/drive/MyDrive/Datasets/Data_separate_files_header_20190329_20200329_8964_yzBR_20210615"
ismn_data = ISMN_Interface(data_path, parallel=True)

In [ ]:
ismn_data.collection

ARM: ['Tryon'],
COSMOS: ['Barrow-ARM', 'Bondville', 'Bushland', 'CC_Pasture', 'CPTEC_INPE', 'Coastal_Sage_UCI', 'Daniel_Forest', 'Desert_Chaparral_UCI', 'Flag_Ponderosa_Pine', 'Flag_Wildfire', 'Fox', 'Freeman_Ranch', 'Goodwin_Creek', 'Harvard_Forest', 'Howland', 'Iowa_Validation_Site', 'KLEE', 'Lower_Salt_Creek', 'Manitou_Forest_Ground', 'Manitou_Forest_Tower', 'Marshall_Colorado', 'Metolius', 'Mozark', 'Mpala_North', 'NJ_Meadowlands', 'Neb_Field_3', 'P301', 'Park_Falls', 'Pe-de-Gigante', 'Reynolds_Creek', 'Rietholzbach', 'SMAP-OK', 'Savannah_River', 'Shale_Hills', 'Silver_Sword', 'Tenderfoot_Creek', 'Tonzi_Ranch', 'Tower_Ruin', 'Tullochgorum', 'Weany_Creek', 'York_Irrigated_Maize', 'York_Irrigated_Soybean', 'York_Rainfed'],
FLUXNET-AMERIFLUX: ['Bouldin_Island_Alfalfa', 'Bouldin_Island_corn', 'Sherman_Barn', 'Tonzi_Ranch', 'Vaira_Ranch'],
FMI: ['SAA111', 'SAA112', 'SAA120', 'SOD011', 'SOD012', 'SOD013', 'SOD021', 'SOD022', 'SOD023', 'SOD071', 'SOD072', 'SOD073', 'SOD081', 'SOD082', 'SO

In [ ]:
def get_name(station):
  return station.metadata.to_dict()["station"][0][0]
  
for key,i in enumerate(ismn_data.collection['USCRN']):
  print(key,get_name(i))



In [ ]:
 ismn_data.collection['REMEDHUS']['El_Tomillar'].metadata

In [ ]:
 ismn_data.collection['REMEDHUS']['El_Tomillar']

Sensors at 'El_Tomillar': ['Stevens-Hydra-Probe_soil_temperature_0.000000_0.050000', 'Stevens-Hydra-Probe_soil_moisture_0.000000_0.050000']

In [ ]:
sensor1 = ismn_data.collection['USCRN']['Champaign_9_SW']['Stevens-Hydraprobe-II-Sdi-12_soil_moisture_0.050000_0.050000']
sensor2 = ismn_data.collection['USCRN']['Champaign_9_SW']['Stevens-Hydraprobe-II-Sdi-12_soil_moisture_0.200000_0.200000']
ts1 = sensor1.read_data()
ts2 = sensor2.read_data()

ts1["depth"] = float(str('Stevens-Hydraprobe-II-Sdi-12_soil_moisture_0.050000_0.050000').split("_")[-1])

ts2["depth"] = float(str('Stevens-Hydraprobe-II-Sdi-12_soil_moisture_0.200000_0.200000').split("_")[-1])
  
ts1.drop('soil_moisture_orig_flag', inplace=True, axis=1)  
ts1.drop('soil_moisture_flag', inplace=True, axis=1)  
ts1.rename(columns={'soil_moisture': 'soil_moisture_depth_0.O5'},inplace=True)
ts2.drop('soil_moisture_orig_flag', inplace=True, axis=1)  
ts2.drop('soil_moisture_flag', inplace=True, axis=1)  
ts2.rename(columns={'soil_moisture': 'soil_moisture_depth_0.O5'},inplace=True)

print(ts2) 
print(ts1) 

                     soil_moisture_depth_0.O5  depth
date_time                                           
2019-03-29 00:00:00                     0.409    0.2
2019-03-29 01:00:00                     0.408    0.2
2019-03-29 02:00:00                     0.418    0.2
2019-03-29 03:00:00                     0.429    0.2
2019-03-29 04:00:00                     0.434    0.2
...                                       ...    ...
2020-03-29 19:00:00                     0.404    0.2
2020-03-29 20:00:00                     0.404    0.2
2020-03-29 21:00:00                     0.406    0.2
2020-03-29 22:00:00                     0.404    0.2
2020-03-29 23:00:00                     0.402    0.2

[8776 rows x 2 columns]
                     soil_moisture_depth_0.O5  depth
date_time                                           
2019-03-29 00:00:00                     0.445   0.05
2019-03-29 01:00:00                     0.463   0.05
2019-03-29 02:00:00                     0.475   0.05
2019-03-29 03:00:00  

In [ ]:
L=['Stevens-Hydra-Probe_soil_moisture_0.000000_0.050000']

In [ ]:

def get_sensor_by(network, station):
  df = []
  
  for index,value in enumerate(ismn_data.collection[network][station]) :
    #print("soil_moisture" in str(value),index)
    #print("--------------")
    sensor = ismn_data.collection[str(network)][str(station)][str(value)]
    
    if str(value) in L :

      ts = sensor.read_data()
      ts.drop('soil_moisture_orig_flag', inplace=True, axis=1)  
      ts.drop('soil_moisture_flag', inplace=True, axis=1)  
      ts["depth"] = float(str(value).split("_")[-1])
      ts.rename(columns={'soil_moisture': 'soil_moisture_depth_'+str(value).split("_")[-1]},inplace=True)
      df.append(ts)
      allS = pd.concat(df)

  
  return allS

def get_station_by(network):
  df1 = []
  for index,key in enumerate(ismn_data.collection[network]) :
    #print(get_name(key))
    data = get_sensor_by(network, get_name(key))
    data["station"] = get_name(key)
    df1.append(data)

  allN = pd.concat(df1)
  allN.head()
  return allN 



In [ ]:
dataset


In [ ]:
dataset = get_sensor_by('REMEDHUS','El_Tomillar')

In [ ]:
pd.set_option("display.max_rows", False)
dataset["depth"].unique()

array([0.05])

In [ ]:
dataset.to_csv ('sensors_dataset2.csv', header=True)

In [ ]:
df = pd.read_csv('sensors_dataset2.csv')
df.tail()

,date_time,soil_moisture_depth_0.050000,depth
8517,2020-03-29 19:00:00,0.038,0.05
8518,2020-03-29 20:00:00,0.037,0.05
8519,2020-03-29 21:00:00,0.036,0.05
8520,2020-03-29 22:00:00,0.035,0.05
8521,2020-03-29 23:00:00,0.033,0.05


## 2- Satellite Imagery :

In [ ]:
import ee
import folium
from IPython.display import Image
from datetime import datetime
from datetime import timedelta

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=OyJ7RHI6LrdMQQjAldlW4xW2Ut-8SpNtX-BMjJbGcaI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWh2L7YdylJSxgU2Uyw4ozYwdd3Se71oRl8_qoLeyj7W9RDthBKW_Dc

Successfully saved authorization token.


In [ ]:
# Initialize the library.
ee.Initialize()

### Selection of a the region

In [ ]:
import geemap

In [ ]:
lat1 =  -5.552694516934068
long1 = 41.352322323917655
lat2 =  -5.539422681928112
long2 = 41.35233183784646
lat3 = -5.54018
long3 = 41.34386
lat4 = -5.5535516325646235
long4 = 41.34410053944169

geo = [[lat1, long1], [lat2, long2], [lat3, long3], [lat4, long4]]
aoi = ee.Geometry.Polygon([[[lat1, long1], [lat2, long2], [lat3, long3], [lat4, long4]]])

CLOUD_FILTER = 60

time_range=['2019-03-29', '2020-03-29']

sentinel_filtered = (ee.ImageCollection('COPERNICUS/S2_SR').
                         filterBounds(aoi).
                         select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12']).
                         filterDate(time_range[0], time_range[1]).
                         filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER))
                         )


geemap.ee_export_image_collection_to_drive(sentinel_filtered, folder='export', scale=10, region=aoi)



### Crop Image & save it


In [ ]:
crop_size = 10
np_img = [] 
path =  "/content/drive/MyDrive/test/20190401T163901_20190401T165214_T16TCK.tif"
src_ds = gdal.Open( path)
# data=src_ds.ReadAsArray()
for iBand in range(1, src_ds.RasterCount + 1):
     #print(iBand)
     inband = src_ds.GetRasterBand(iBand)
     cropped_inband = inband.ReadAsArray(inband.XSize/2+crop_size/2,inband.XSize/2-crop_size/2,crop_size,crop_size)
    # print( cropped_inband.shape)
     np_img.append(cropped_inband)
np.array(np_img).shape

(12, 10, 10)

In [ ]:
def crop_img(path,crop_size  ):

  #crop_size = 
  np_img = []
  #path =  "/content/drive/MyDrive/test/20190401T163901_20190401T165214_T16TCK.tif"
  src_ds = gdal.Open( path)

  for iBand in range(1, src_ds.RasterCount + 1):
     #print(iBand)
     inband = src_ds.GetRasterBand(iBand)
     cropped_inband = inband.ReadAsArray(inband.XSize/2+crop_size/2,inband.XSize/2-crop_size/2,crop_size,crop_size)
    # print( cropped_inband.shape)
     np_img.append(cropped_inband)
  return np.array(np_img)

In [ ]:
directory = '/content/drive/MyDrive/test'
data_list =[]
for filename in os.listdir(directory):
    if filename.endswith(".tif"):
        path =  "/content/drive/MyDrive/test/"+str(filename)
        test = crop_img(path,720)
        file_name = path.split('/')[-1].split('.tif')[0]
        store = "/content/drive/MyDrive/dump_test/"
        pth = store+file_name+'.npy'
        with open(pth, 'wb') as f:
            np.save(f, test)
            # np.save(f, np.array([1, 3]))

# 3- Dataset :
Synchronize the field measurement with images dates

In [ ]:
## Requirements :

from osgeo import gdal
import sys
import os

In [ ]:
## Read the data from the 2 stations :

df1 = pd.read_csv("/content/drive/MyDrive/Datasets/sensors_dataset.csv")
df2 = pd.read_csv("/content/drive/MyDrive/Datasets/sensors_dataset2.csv")

In [ ]:
df1.head() 

In [ ]:
df2.head()

In [ ]:
df1.columns 

In [ ]:
df1.describe()

In [ ]:
df2.describe()

In [ ]:
## split dates and hours :

df1['date_time'][1].split(" ") 

In [ ]:
## Add hours column 

df1["hour"] = ""
df2["hour"] = ""

for i in range (8278):
  df1["hour"][i]=df1['date_time'][i].split(" ")[1]
  df1["date_time"][i]=df1['date_time'][i].split(" ")[0]
  
 
for i in range (8522):
  df2["hour"][i]=df2['date_time'][i].split(" ")[1]
  df2["date_time"][i]=df2['date_time'][i].split(" ")[0]

In [ ]:
df1.head()

In [ ]:
test_dataset = pd.DataFrame(columns=['date_time', 'soil_moisture_depth_0.05','hour'])
Rdataset = pd.DataFrame(columns=['date_time','hour', 'soil_moisture_depth_0.05','Image_name'])

In [ ]:
test_dataset['soil_moisture_depth_0.05'] = (df1['soil_moisture_depth_0.050000']+df2['soil_moisture_depth_0.050000'])/2
test_dataset['date_time'] = df1['date_time']
test_dataset['hour'] = df1['hour']
test_dataset.head()

In [ ]:
## Selection of 5pm as the time of each image :

Rdataset = test_dataset[test_dataset["hour"]=='17:00:00']
Rdataset.describe()

In [ ]:
## Selection of the only useful days and add images names :

directory = '/content/drive/MyDrive/Sentinel_images'
data_list =[]
Vals = []

for filename in os.listdir(directory):
    if filename.endswith("QF.tif"):

      im_date = filename[0:4]+"-"+filename[4:6]+"-"+filename[6:8]

      if im_date in Rdataset["date_time"].values : 
          index_date= np.where(Rdataset["date_time"]==im_date )
          rdata = Rdataset.iloc[index_date]
          data_list.append(rdata)
          dataset = pd.concat(data_list)
          Vals.append(filename)


dataset['Image_name']= Vals
dataset.head()
      

In [ ]:
## Save dataset :

dataset.to_csv ('TheDataset.csv', header=True, index=False)